In [1]:
import pandas as pd
import numpy as np
import math
import nltk
from nltk.corpus import stopwords
import re
import string
from collections import Counter
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('books_authors_final.csv')

In [3]:
df

,Unnamed: 0,title,authors,description,number of pages,average rating,genres,reviews' keywords,more about author(s),image_url
0,2,To Kill a Mockingbird,Harper Lee,*Amazon best selling book 2009-2019* The u...,324 pages,4.27 out of 79450 reviews,Classics | Fiction | Historical | Histor...,*amazing* *american* *best* *black* *boy* ...,*Harper Lee*: gender: male | Country: United K...,https://images.gr-assets.com/books/1361975680l...
1,32,Anne of Green Gables,L.M. Montgomery,As soon as Anne Shirley arrives at the snug wh...,320 pages,4.24 out of 16797 reviews,Classics | Fiction | Young Adult | Child...,*amazing* *bad* *better* *certain* *classi...,*L.M. Montgomery*: gender: unknown | average r...,https://images.gr-assets.com/books/1390789015l...
2,117,Tuesdays with Morrie,Mitch Albom|Saulius Dagys,"Maybe it was a grandparent, or a teacher or a ...",210 pages,4.08 out of 21367 reviews,Nonfiction | Autobiography | Memoir | Bi...,*enjoy* *everything* *final* *good* *great...,*Mitch Albom*: website: http://www.nannafoss.d...,https://images.gr-assets.com/books/1423763749l...
3,141,The Divine Comedy,Dante Alighieri|Allen Mandelbaum|Eugenio Montale,The Divine Comedy describes Dante's descent in...,798 pages,4.07 out of 3127 reviews,Classics | Poetry | Fiction | Literature,*anything* *better* *close* *different* *d...,*Dante Alighieri*: website: http://www.terri.c...,https://images.gr-assets.com/books/1390760180l...
4,142,The Compleat Works of Wllm Shkspr (abridged),Reduced Shakespeare Company|Jess Borgeson|Adam...,Revised from the rather long original complete...,137 pages,4.47 out of 127 reviews,Plays | Classics | Humor | Fiction | D...,*better* *complete* *funny* *good* *great*...,*Reduced Shakespeare Company*: gender: unknown...,https://images.gr-assets.com/books/1347579233l...
...,...,...,...,...,...,...,...,...,...,...
4965,54191,Still Waters,Emma Carlson Berne,Hannah can't wait to sneak off for a romantic ...,212 pages,3.17 out of 146 reviews,Young Adult | Mystery | Horror | Thriller | Ro...,*bad* *complete* *dark* *enough* *everythi...,*Emma Carlson Berne*: gender: male | average r...,https://images.gr-assets.com/books/1327985610l...
4966,54239,Angels Everywhere,Debbie Macomber,A Heavensent Gift of Love In a season of givin...,640 pages,4.25 out of 62 reviews,Romance | Holiday | Christmas | Fiction,*first* *funny* *good* *great* *happy* *h...,*Debbie Macomber*: gender: unknown | average r...,https://images.gr-assets.com/books/1348822196l...
4967,54240,The Royal Family,William T. Vollmann,A magnificent new novel from a writer whose bo...,774 pages,3.89 out of 82 reviews,Fiction | Novels | Contemporary | Mystery | Crime,*american* *anything* *better* *early* *en...,*William T. Vollmann*: website: https://philli...,https://images.gr-assets.com/books/1386921561l...
4968,54256,My Pony,Susan Jeffers,I want a pony. I want a pony more than anythin...,32 pages,4.08 out of 38 reviews,Childrens | Picture Books | Animals | Horses |...,*anything* *beautiful* *become* *favorite* ...,*Susan Jeffers*: gender: male | Country: Unite...,https://images.gr-assets.com/books/1387668516l...


In [4]:
book_titles_dict = df['title'].to_dict()
opt = []
for k,v in book_titles_dict.items():
    temp_d = {}
    temp_d["label"] = v
    temp_d["value"] = v
    opt.append(temp_d)
    

In [5]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_sim(*strs): 
#     print(strs)
    vectors = [t for t in get_vectors(*strs)]
    return cosine(*vectors)

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
    
def get_vectors(*strs):
    text = []
    for t in strs:
        t = t.translate(str.maketrans('', '', string.punctuation))
        stop_words = stopwords.words('english')
        stopwords_dict = Counter(stop_words)
        text.append(' '.join([word for word in t.split() if word not in stopwords_dict]))
    
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [16]:
def recommend_desc(book):
    #user input
    if len(df[df['title'] == book.lower()]) > 0:
        desc = list(df[df['title'] == book.lower()]['description'])[0]
        print('Found match: ', book, '\n')
        match = book
    else:
        index = np.argmax([fuzz.ratio(book.lower(), i) for i in list(df['title']) if type(i)== str])
        desc = df.iloc[index,:]['description']
        print('Found closest match: ', df.iloc[index,:]['title'], '\n')
        match = df.iloc[index,:]['title']
    
    all_desc = list(df['description'])
    all_genres = list(df['genres'])
    similarity_array = np.zeros([len(all_desc),])
    genre_input = df.iloc[df[df['title'] == 'To Kill a Mockingbird'.lower()].index[0],:]['genres'].split('|')
    for k, i in enumerate(all_desc):
        if len(list(set(genre_input) & set(all_genres[k].split('|')))) > 2:
            if type(i) == str:
                value = get_cosine_sim(i, desc)
                if value != math.nan:
                    similarity_array[k] = get_cosine_sim(desc, i)
    similarity = similarity_array.tolist()
    similarity.remove(max(similarity))
    final_index = np.nanargmax(similarity)
    
    response = requests.get(df.iloc[final_index,:]['image_url'])

    img = Image.open(BytesIO(response.content))

    return [match, df.iloc[final_index,:]['title'], df.iloc[final_index,:]['description'], 
            df.iloc[final_index,:]['average rating'], df.iloc[final_index,:]['number of pages'], df.iloc[final_index,:]['authors'], df.iloc[final_index,:]['image_url']]


In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([html.H1('Keyword Recommendation',
            style={"background-color": "coral","margin-left":"auto","margin-right":"auto", 'display': 'flex', 'justify-content': 'center'}), html.Br(), 
    dcc.Dropdown(
        id='demo-dropdown',
        options= opt,
        value='',
        style={"width":"1300px"}
    ), html.Br(),
    html.Div(html.Center(id='url')),
    html.Div(
        html.Table([
        html.Tr([html.Th(html.H4('Top Recommendation:',style={"color": "white","width":"300px"}),style={"background-color": "grey"}), 
                 html.Td(id='top-rec',style={"width":"500px"})], style={"border": "2px solid grey"}),
        html.Tr([html.Th(html.H6('Author:',style={"color": "white"}),style={"background-color": "grey"}), html.Td(id='author')], style={"border": "2px solid grey"}),
        html.Tr([html.Th(html.H6('Rating:',style={"color": "white"}),style={"background-color": "grey"}), html.Td(id='rating')], style={"border": "2px solid grey"}),
        html.Tr([html.Th(html.H6('Length:',style={"color": "white"}),style={"background-color": "grey"}), html.Td(id='length')], style={"border": "2px solid grey"}),
    ], style={"border-spacing": "60px","border": "2px solid grey",
              "margin-left":"500px","margin-right":"auto","background-color": "coral", "opacity": "0.9"}
    )),
    html.Div(html.Table([
        html.Tr([html.Td(html.H6('Description')), html.Td(id='desc')]),
    ], style={"width": "83%", }))])


@app.callback(
    Output('top-rec', 'children'),
    Output('author', 'children'),
    Output('desc', 'children'),
    Output('rating', 'children'),
    Output('length', 'children'),
    Output('url', 'children'),
    [dash.dependencies.Input('demo-dropdown', 'value')])

def update_output(value):
    x = recommend_desc(value)

    # response = requests.get(x[6])

    img = html.Img(src= x[6],style={"float":"left","width": "300px", "height": "400px" })
    # recc = type(x[1])
    #"max-width": "240px","max-height": "346px", "position": "relative", "top": "200px", "left": "0px", "width": "350.971px", "height": "400.667px"
    l = '{} pages.'.format(str(int(x[4])))

    return x[1],  x[5],  x[2], x[3],l, img


if __name__ =='__main__':
    app.run_server(port=4000)

Dash is running on http://127.0.0.1:4000/

Dash is running on http://127.0.0.1:4000/

Dash is running on http://127.0.0.1:4000/

Dash is running on http://127.0.0.1:4000/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2021 23:50:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 23:50:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 23:50:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 23:50:40] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -


Found closest match:  To Kill a Mockingbird 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/dash/dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/da

127.0.0.1 - - [27/May/2021 23:50:40] "POST /_dash-update-component HTTP/1.1" 500 -


Found closest match:  Anne of Green Gables 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/dash/dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/das

...

127.0.0.1 - - [28/May/2021 00:00:21] "POST /_dash-update-component HTTP/1.1" 500 -


Found closest match:  The Sneetches and Other Stories 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/dash/dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-p

127.0.0.1 - - [28/May/2021 00:00:23] "POST /_dash-update-component HTTP/1.1" 500 -


Found closest match:  Kindred 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/dash/dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/sepehr/opt/miniconda3/lib/python3.7/site-packages/dash/dash.py", l

127.0.0.1 - - [28/May/2021 00:04:43] "POST /_dash-update-component HTTP/1.1" 500 -
